In [1]:
import random
from math import ceil

### Initialise

In [2]:
minor = [0,2,3,5,7,8,10,12]
target = [0, 2, 3, 5, 0, 2, 3, 8]
# target = [12, 12, 12, 12, 5, 7, 0, 12]

In [3]:
agents = [[minor[random.randint(0,7)] for x in range(8)] for i in range(100)]

In [4]:
agents[:4]

[[8, 12, 5, 3, 8, 7, 0, 0],
 [0, 0, 10, 12, 0, 12, 2, 3],
 [3, 12, 10, 2, 10, 2, 2, 12],
 [7, 0, 2, 0, 8, 12, 10, 10]]

In [40]:
", ".join([str(i) for i in agents[0]])

'3, 7, 5, 0, 5, 5, 5, 12'

In [50]:
from collections.abc import Sequence


class Agent(Sequence):
    def __init__(self, target, scale=minor, notes=None):
        self.target = target
        self.scale = scale
        self.n = len(target) 
        if not notes:
            self.notes = [self.scale[random.randint(0, self.n - 1)] for i in range(0, self.n)]
        else:
            self.notes = notes
        super().__init__()
    
    def __repr__(self) -> str:
        return f"[{', '.join([str(i) for i in self.notes])}]"
    
    def __getitem__(self, i) -> int:
        return self.notes[i]
    
    def __len__(self) -> int:
        return len(self.notes)   

    def __eq__(self, other) -> bool:
        if len(self.notes) != len(other.notes):
            return False
        return all(self.notes[idx]==other.notes[idx] for idx in range(0, self.n))
            
    def __le__(self, other) -> bool:
        return self.evaluate() >= other.evaluate()

    def __lt__(self, other) -> bool:
        return self.evaluate() > other.evaluate()

    def __ge__(self, other) -> bool:
        return self.evaluate() <= other.evaluate()

    def __gt__(self, other) -> bool:
        return self.evaluate() < other.evaluate()
    
    def set_target(self, target):
        self.target = target
    
    # def reproduce(self, partner) -> Agent:
    #     crossover = random.randint(1, self.n - 1)
    #     return Agent(
    #         target=self.target,
    #         scale=self.scale,
    #         notes=self.notes[:crossover] + partner.notes[crossover:]
    #     )
    
    def reproduce(self, partner) -> "Agent":
        crossover_mask = [random.randint(0,1) for i in range(len(self.notes))]
        inheritance = {0: self.notes, 1: partner.notes}
        return Agent(
            target=self.target,
            scale=self.scale,
            notes=[inheritance[x][n] for n,x in enumerate(crossover_mask)]
        )
        
    # def _evaluate(self) -> int:
    #     return sum([(self.target[idx] - self.notes[idx]) ** 2 for idx in range(self.n-1)])
    
    # def evaluate(self):
    #     score = self._evaluate()
    #     transition_factor=0
    #     target_transitions = [(self.target[i], self.target[i+1]) for i in range(self.n - 1)]
    #     note_transitions = [(self.notes[i], self.notes[i+1]) for i in range(self.n - 1)]
    #     for note_transition, target_transition in zip(note_transitions, target_transitions):
    #         if note_transition==target_transition:
    #             score+=1
    #     return score * (1 - (transition_factor / self.n))
    
    def evaluate(self):
        score=0
        target_transitions = [(self.target[i], self.target[i+1]) for i in range(self.n - 1)] + [(self.target[-1], self.target[0])]
        note_transitions = [(self.notes[i], self.notes[i+1]) for i in range(self.n - 1)] + [(self.notes[-1], self.notes[0])]
        for note_transition, target_transition in zip(note_transitions, target_transitions):
            if note_transition==target_transition:
                score+=1
        return score
    
    def mutate(self) -> None:
        idx_to_mutate = random.randint(0, self.n-1)
        note_to_mutate = self.notes[idx_to_mutate]
        self.notes[idx_to_mutate] = random.choice(list(set(self.scale) - {note_to_mutate}))
    
        
class Population():
    def __init__(self, target, r: float = 0.1, m: float = 0.2, n_agents: int = 20, scale: list[int] = minor):
        self.reproduction_rate = r
        self.mutation_rate = m
        self.n_agents = n_agents
        self.target = target
        self.scale = scale 
        self.agents = [Agent(target=self.target, scale=self.scale) for idx in range(0,self.n_agents)]
        self.n_generations = 0
    
    def __repr__(self):
        return f"Population with {self.n_agents} agents and targetting: {target}"
    
    def __getitem__(self, i):
        return self.agents[i]
    
    def __len__(self):
        return self.n_agents   

    def __iter__(self):
        for agent in self.agents:
            yield agent

    def process(self):
        fitnesses = [agent.evaluate() for agent in self.agents]
        selection_weights = [f / sum(fitnesses) for f in fitnesses]
        # selection_weights = [1 - (f / sum(fitnesses)) for f in fitnesses]
        best_agent = min(self.agents)
        
        print(f"{self.n_generations} Best agent: {best_agent}: {best_agent.evaluate()}")
        not_dead = random.choices(
            self.agents,
            k=int((1-self.reproduction_rate) * self.n_agents),
            weights=selection_weights, 
        )
        n_to_reproduce = (self.n_agents - len(not_dead)) * 2
        
        reproduction_pool = random.shuffle(
            random.choices(
                self.agents, 
                k=n_to_reproduce,
                weights=selection_weights,
            )
        )
        new_offspring = []
        for i in range(0,int(n_to_reproduce/2)):
            new_offspring.append(
                self.agents[i].reproduce(self.agents[i+int(n_to_reproduce/2)])
            )
        
        self.agents = not_dead + new_offspring
        
        for idx in random.sample(range(0, self.n_agents), k=int(self.mutation_rate * self.n_agents)):
            self.agents[idx].mutate()
        self.n_generations += 1
        

In [ ]:
class Population():
    def __init__(self, target, r: float = 0.1, m: float = 0.2, n_agents: int = 20, scale: list[int] = minor):
        self.reproduction_rate = r
        self.mutation_rate = m
        self.n_agents = n_agents
        self.target = target
        self.scale = scale 
        self.agents = [Agent(target=self.target, scale=self.scale) for idx in range(0,self.n_agents)]
        self.n_generations = 0
    
    def __getitem__(self, i):
        return self.agents[i]
    
    def __len__(self):
        return self.n_agents   


    def process(self):
        fitnesses = [agent.evaluate() for agent in self.agents]
        selection_weights = [f / sum(fitnesses) for f in fitnesses]
        best_agent = min(self.agents)
        
        # Randomly (weighted by fitness) select (with replacement) agents that survive this iteration
        not_dead = random.choices(
            self.agents,
            k=int((1-self.reproduction_rate) * self.n_agents),
            weights=selection_weights, 
        )
        
        # calculate the number of agents that should reproduce
        n_to_reproduce = (self.n_agents - len(not_dead)) * 2
        
        # Weighted by fitness) select (with replacement) n_to_reproduce agents
        reproduction_pool = random.choices(
            self.agents, 
            k=n_to_reproduce,
            weights=selection_weights,
        )
    
        # make pairs of offspring from the reproduction pool, and reproduce to create new_offspring
        new_offspring = []
        for i in range(0, int(n_to_reproduce/2)):
            new_offspring.append(
                reproduction_pool[i].reproduce(reproduction_pool[i+int(n_to_reproduce/2)])
            )
            
        # the next iteration's agents are those surviving agents and the offspring of those that reproduced
        self.agents = not_dead + new_offspring
        
        # Mutate the proportion of agents dictated by the mutation rate
        for idx in random.sample(range(0, self.n_agents), k=int(self.mutation_rate * self.n_agents)):
            self.agents[idx].mutate()
        self.n_generations += 1

In [51]:
population = Population(target=target, r=0.25, m=0.05, n_agents=750)
for i in range(20):
    population.process()

0 Best agent: [12, 2, 3, 5, 2, 0, 7, 3]: 2
1 Best agent: [5, 2, 3, 5, 12, 7, 3, 8]: 3
2 Best agent: [0, 2, 3, 5, 2, 7, 3, 8]: 5
3 Best agent: [8, 2, 3, 5, 0, 2, 3, 2]: 5
4 Best agent: [8, 2, 3, 5, 0, 2, 3, 2]: 5
5 Best agent: [0, 2, 3, 5, 7, 8, 3, 8]: 5
6 Best agent: [0, 2, 3, 5, 0, 0, 10, 8]: 5
7 Best agent: [0, 2, 3, 5, 0, 2, 3, 8]: 8
8 Best agent: [0, 2, 3, 5, 0, 2, 3, 8]: 8
9 Best agent: [0, 2, 3, 5, 0, 2, 3, 8]: 8
10 Best agent: [0, 2, 3, 5, 0, 2, 3, 8]: 8
11 Best agent: [0, 2, 3, 5, 0, 2, 3, 8]: 8
12 Best agent: [0, 2, 3, 5, 0, 2, 3, 8]: 8
13 Best agent: [0, 2, 3, 5, 0, 2, 3, 8]: 8
14 Best agent: [0, 2, 3, 5, 0, 2, 3, 8]: 8
15 Best agent: [0, 2, 3, 5, 0, 2, 3, 8]: 8
16 Best agent: [0, 2, 3, 5, 0, 2, 3, 8]: 8
17 Best agent: [0, 2, 3, 5, 0, 2, 3, 8]: 8
18 Best agent: [0, 2, 3, 5, 0, 2, 3, 8]: 8
19 Best agent: [0, 2, 3, 5, 0, 2, 3, 8]: 8


In [42]:
target

[0, 2, 3, 5, 0, 2, 3, 8]

In [43]:
for i in range(20):
    population.process()

0 Best agent: [0, 2, 3, 5, 10, 5, 2, 10]: 4
1 Best agent: [0, 2, 3, 5, 10, 5, 2, 10]: 4
2 Best agent: [0, 2, 3, 5, 10, 5, 2, 10]: 4
3 Best agent: [0, 2, 3, 5, 10, 5, 2, 10]: 4
4 Best agent: [0, 2, 3, 5, 10, 5, 7, 10]: 4
5 Best agent: [0, 2, 3, 5, 0, 5, 7, 10]: 5
6 Best agent: [0, 2, 3, 5, 0, 5, 7, 10]: 5
7 Best agent: [0, 2, 3, 5, 0, 5, 7, 8]: 6
8 Best agent: [0, 2, 3, 5, 0, 5, 7, 8]: 6
9 Best agent: [0, 2, 3, 5, 10, 5, 7, 8]: 5
10 Best agent: [0, 2, 3, 5, 8, 12, 8, 10]: 4
11 Best agent: [0, 2, 3, 5, 10, 5, 7, 10]: 4
12 Best agent: [0, 2, 3, 5, 8, 5, 2, 7]: 4
13 Best agent: [0, 2, 3, 5, 10, 12, 7, 12]: 4
14 Best agent: [0, 2, 3, 5, 0, 12, 10, 12]: 5
15 Best agent: [0, 2, 3, 5, 3, 5, 2, 8]: 5
16 Best agent: [0, 2, 3, 5, 3, 5, 2, 8]: 5
17 Best agent: [0, 2, 3, 5, 10, 5, 5, 10]: 4
18 Best agent: [0, 2, 3, 5, 10, 5, 7, 10]: 4
19 Best agent: [0, 2, 3, 5, 5, 3, 2, 10]: 4


In [450]:
blah = [3, 2, 2, 5, 2, 2, 3, 3]
# random.shuffle(blah)
blah

[3, 2, 2, 5, 2, 2, 3, 3]

In [375]:
xx=[(blah[i],blah[i+1]) for i in range(len(blah)-1)]

In [376]:
yy=[(target[i],target[i+1]) for i in range(len(target)-1)]

In [380]:
score=0
for x,y in zip(xx,yy):
    if x==y:
        score+=1
1-(score/len(yy))

0.8571428571428572

In [221]:
random.choices(k=2, weights=selection)

TypeError: Random.choices() missing 1 required positional argument: 'population'

In [197]:
[a.notes[idx]==b.notes[idx] for idx in range(0, len(target)-1)]

[False, False, True, False, False, False, False]

In [198]:
[a.notes[idx]==a.notes[idx] for idx in range(0, len(target)-1)]

[True, True, True, True, True, True, True]

In [202]:
all(a.notes[idx]==b.notes[idx] for idx in range(0, len(target)-1)), all(a.notes[idx]==a.notes[idx] for idx in range(0, len(target)-1))

(False, True)

In [189]:
for n, agent in enumerate(population):
    print(n,agent)

0 [12, 2, 5, 7, 3, 8, 10]
1 [8, 8, 5, 3, 10, 10, 0]
2 [5, 0, 12, 7, 10, 7, 2]
3 [3, 8, 7, 10, 5, 10, 7]
4 [7, 10, 5, 5, 5, 8, 12]
5 [7, 12, 2, 3, 10, 2, 8]
6 [8, 0, 10, 10, 8, 8, 2]
7 [5, 5, 0, 3, 12, 8, 8]
8 [3, 5, 8, 12, 0, 8, 5]
9 [5, 10, 7, 3, 0, 12, 7]
10 [3, 2, 0, 7, 5, 10, 5]
11 [3, 3, 8, 3, 5, 0, 5]
12 [12, 3, 8, 8, 0, 3, 10]
13 [7, 3, 5, 10, 12, 2, 7]
14 [5, 2, 7, 10, 5, 0, 2]
15 [12, 0, 5, 3, 7, 0, 5]
16 [12, 10, 8, 8, 0, 3, 5]
17 [10, 7, 0, 2, 7, 8, 5]
18 [0, 5, 7, 3, 12, 10, 5]
19 [12, 8, 8, 3, 7, 5, 0]
20 [0, 5, 8, 2, 0, 5, 10]
21 [7, 3, 12, 3, 10, 10, 8]
22 [2, 2, 8, 12, 10, 5, 12]
23 [0, 2, 2, 10, 3, 7, 2]
24 [2, 3, 10, 0, 5, 5, 12]
25 [0, 0, 12, 0, 12, 2, 8]
26 [12, 0, 5, 7, 7, 12, 0]
27 [0, 12, 0, 12, 0, 0, 2]
28 [8, 0, 10, 2, 2, 5, 7]
29 [12, 10, 10, 8, 10, 5, 5]


In [222]:
a = Agent(target=target)
b = Agent(target=target)
c = Agent(target=target)
d = Agent(target=target)
e = a.reproduce(b)
f = c.reproduce(a)
g = Agent(target=target)
agents = (a,b,c,d,e,f,g)
fitness = [agent.evaluate() for agent in agents]
total_fitness = sum(fitness)
selection = [1-(f/total_fitness) for f in fitness]
print(list(zip(fitness, selection)))

[(77, 0.9292929292929293), (48, 0.9559228650137741), (128, 0.8824609733700642), (396, 0.6363636363636364), (94, 0.9136822773186409), (116, 0.8934802571166207), (230, 0.7887970615243343)]


In [223]:
random.choices(agents, k=2, weights=selection)

[[0, 3, 5, 8, 5, 5, 3, 12], [7, 3, 0, 8, 5, 3, 3, 12]]

In [206]:
a>b 

True

In [154]:
print(b)
b.mutate()
print(b)
b[-3:]

[10, 5, 12, 2, 0, 12, 5]
[10, 5, 12, 2, 0, 12, 7]


[0, 12, 7]

### Perform

In [17]:
def fitness(agent: list[int]) -> float:
    return sum([target[n] - i for n,i in enumerate(agent)])/8

In [26]:
fitnesses = [fitness(agent) for agent in agents]
agents[fitnesses.index(max(fitnesses))]

[0, 0, 2, 2, 7, 10, 2, 2]

### Select

### Regenerate